In [1]:
import os
import shutil
import pandas as pd
from glob import glob
from joblib import Parallel, delayed
# This script only keep every 15th frame of the dataset
# Nonedestructive and only creates a copy in a new folder
keep_every = 15
ram = True
source_path = '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/full'
destination = '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames'
ls_destination = destination
if ram:
    if not os.path.exists('/home/smoothjazzuser/Desktop/ram/frames/'):
        os.makedirs('/home/smoothjazzuser/Desktop/ram/frames/')
    destination = '/home/smoothjazzuser/Desktop/ram/frames'
    !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/
    !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/Desktop/ram/frames/
    !rm -rf /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
else: 
    !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/
    #del /frames/ folder
    !rm -rf /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
    #create new /frames/ folder
    !mkdir /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
    print('Saving to RAM')
!mkdir -p /tmp/empty


rsync: [sender] change_dir "/tmp/empty" failed: No such file or directory (2)
rsync error: some files/attrs were not transferred (see previous errors) (code 23) at main.c(1327) [sender=3.2.5]
rsync: [sender] change_dir "/tmp/empty" failed: No such file or directory (2)
rsync error: some files/attrs were not transferred (see previous errors) (code 23) at main.c(1327) [sender=3.2.5]


In [2]:


folders_list = sorted(glob(os.path.join(source_path, '*')))
print(len(folders_list))

# create all desitnation folders, if they don't exist
for folder in folders_list:
    if not os.path.exists(os.path.join(destination, folder.split('/')[-1])):
        os.makedirs(os.path.join(destination, folder.split('/')[-1]))

def copy_frames(folder):
    files = sorted(glob(os.path.join(folder, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
    dest = os.path.join(destination, folder.split('/')[-1])
    for i, file in enumerate(files):
        if i % keep_every == 0:
            shutil.copy(file, os.path.join(dest, os.path.basename(file)))
    # rename all files in the folder to be sequential, starting at 0
    files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
    for i, file in enumerate(files):
        os.rename(file, os.path.join(dest, str(i).zfill(3)+'.jpg'))

    # remove frames until len(files) is a multiple of 5
    files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
    while len(files) % 1 != 0:
        os.remove(files[-1])
        files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))



668


In [3]:
c = Parallel(n_jobs=12)(delayed(copy_frames)(folder) for folder in folders_list)
del c

In [ ]:
# link with shortcut '/home/smoothjazzuser/Desktop/ram/frames' and '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames' to the same folder
if ram:
    !ln -s /home/smoothjazzuser/Desktop/ram/frames/ /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/